<a href="https://colab.research.google.com/github/Tasneem-Ibrahim/Python/blob/main/Project_03_Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function_calling

## 1. Function Calling Config

### Install Google Generative AI Library

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2"

In [ ]:
# Import Necessary Modules
from google.colab import userdata
import google.generativeai as genai

# Configure Google Generative AI API
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

### Set up a model with tools

In [ ]:
def turn_on_uv_lights():
    """Turn on the uv lighting system."""
    print("LIGHTBOT: UV lights enabled.")

def turn_on_light():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def change_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def turn_off_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [turn_on_uv_lights, turn_on_light, change_light_color, turn_off_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

Create a helper function for setting `function_calling_config` on `tool_config`.

In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

### Text-only mode: `NONE`

If you have provided the model with tools, but do not want to use those tools for the current conversational turn, then specify `NONE` as the mode. `NONE` tells the model not to make any function calls, and will behave as though none have been provided.

In [ ]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

I can turn lights on and off, and I can change their color.



### Automatic mode: `AUTO`

To allow the model to decide whether to respond in text or call specific functions, you can specify `AUTO` as the mode.

In [ ]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Turn on the UV lights!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "turn_on_uv_lights"
  args {
  }
}



### Function-calling mode: `ANY`

Setting the mode to `ANY` will force the model to make a function call. By setting `allowed_function_names`, the model will only choose from those functions. If it is not set, all of the functions in `tools` are candidates for function calling.

In this example system, if the lights are already on, then the user can change color or turn the lights off.

In [ ]:
available_fns = ["change_light_color", "turn_off_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place Orange!", tool_config=tool_config)
print(response.parts[0])


function_call {
  name: "change_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "FF6600"
      }
    }
  }
}



### Automatic function calling

`tool_config` works when enabling automatic function calling too.

In [ ]:
available_fns = ["turn_on_light"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
#auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)
auto_chat.send_message("Set light color in yellow...", tool_config=tool_config)

LIGHTBOT: Lights enabled.
LIGHTBOT: Lights set to #FFFF00.


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Lights are now set to yellow. anything else?"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.07512627840042115
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 246,
        "candidates_token_count": 10,
        "total_token_count": 256
      }
    }),
)

In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [ ]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party !")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [ ]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Ok, I've turned on the disco ball, started playing some energetic music (Never gonna give you up), and dimmed the lights to 50% brightness. Let's party!



In [ ]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                             tools=[multiply])

model._tools.to_proto()

[function_declarations {
   name: "multiply"
   description: "returns a * b."
   parameters {
     type_: OBJECT
     properties {
       key: "b"
       value {
         type_: NUMBER
       }
     }
     properties {
       key: "a"
       value {
         type_: NUMBER
       }
     }
     required: "a"
     required: "b"
   }
 }]

### Task

In [ ]:
# prompt: make a list of dictionary , key samples is 10 samples of person, which include name, father name , education and address..

import random

def generate_person_data():
  names = ["Alice", "Bob", "Charlie", "David", "Eve", "Jona", "Alex", "Ema", "Sophia", "William"]
  father_names = ["Robert", "Michael", "James", "John", "David", "Richard", "Joseph", "Thomas", "Charles", "Christopher"]
  educations = ["High School", "Bachelor's", "Master's", "PhD", "Associate's"]
  addresses = ["123 Main St", "456 Oak Ave", "789 Pine Ln", "1011 Elm St", "1213 Maple Dr"]

  name = random.choice(names)
  father_name = random.choice(father_names)
  education = random.choice(educations)
  address = random.choice(addresses)
  return {
      "name": name,
      "father_name": father_name,
      "education": education,
      "address": address
  }


samples = []
for _ in range(10):
  samples.append(generate_person_data())

samples

[{'name': 'Alex',
  'father_name': 'Charles',
  'education': "Associate's",
  'address': '789 Pine Ln'},
 {'name': 'Bob',
  'father_name': 'Charles',
  'education': "Associate's",
  'address': '1213 Maple Dr'},
 {'name': 'Alex',
  'father_name': 'Robert',
  'education': "Associate's",
  'address': '123 Main St'},
 {'name': 'David',
  'father_name': 'James',
  'education': "Master's",
  'address': '456 Oak Ave'},
 {'name': 'Ema',
  'father_name': 'Thomas',
  'education': 'High School',
  'address': '1213 Maple Dr'},
 {'name': 'David',
  'father_name': 'David',
  'education': 'PhD',
  'address': '789 Pine Ln'},
 {'name': 'Charlie',
  'father_name': 'David',
  'education': "Master's",
  'address': '456 Oak Ave'},
 {'name': 'David',
  'father_name': 'David',
  'education': "Bachelor's",
  'address': '1213 Maple Dr'},
 {'name': 'Eve',
  'father_name': 'Thomas',
  'education': 'PhD',
  'address': '1213 Maple Dr'},
 {'name': 'Bob',
  'father_name': 'Joseph',
  'education': "Bachelor's",
  'ad

In [ ]:
# prompt: Make a function with above list when we ask name as a input, it will give information to that person.

def get_person_info(name):
    for person in samples:
        if person["name"] == name:
            return person
    return "No information found for that person."

# Example usage
person_name = input("Enter the name of the person: ")
info = get_person_info(person_name)
info

Enter the name of the person: Bob


{'name': 'Bob',
 'father_name': 'Charles',
 'education': "Associate's",
 'address': '1213 Maple Dr'}

### Task with LLM

In [ ]:
# prompt: make a list of dictionary , key samples is 10 samples of person, which include name, father name , education and address.. include LLM in it

import random

def generate_person_data():
    names = ["Alice", "Bob", "Charlie", "David", "Eve", "Jona", "Sophia", "William", "Olivia", "James"]
    father_names = ["Robert", "Michael", "William", "David", "Richard", "Joseph", "Thomas", "Charles", "Christopher", "Daniel"]
    educations = ["High School", "Bachelor's Degree", "Master's Degree", "PhD", "Associate's Degree"]
    addresses = ["123 Main St", "456 Oak Ave", "789 Pine Ln", "1011 Elm St", "1213 Maple Dr"]

    name = random.choice(names)
    father_name = random.choice(father_names)
    education = random.choice(educations)
    address = random.choice(addresses)

    return {
        "name": name,
        "father_name": father_name,
        "education": education,
        "address": address,
    }

samples = []
for _ in range(10):
    samples.append(generate_person_data())

samples

[{'name': 'Alice',
  'father_name': 'Robert',
  'education': 'PhD',
  'address': '789 Pine Ln'},
 {'name': 'Alice',
  'father_name': 'Daniel',
  'education': "Master's Degree",
  'address': '1011 Elm St'},
 {'name': 'David',
  'father_name': 'Daniel',
  'education': "Master's Degree",
  'address': '1011 Elm St'},
 {'name': 'James',
  'father_name': 'David',
  'education': 'PhD',
  'address': '789 Pine Ln'},
 {'name': 'Sophia',
  'father_name': 'William',
  'education': "Associate's Degree",
  'address': '456 Oak Ave'},
 {'name': 'David',
  'father_name': 'Charles',
  'education': 'High School',
  'address': '123 Main St'},
 {'name': 'Bob',
  'father_name': 'Christopher',
  'education': 'High School',
  'address': '1213 Maple Dr'},
 {'name': 'William',
  'father_name': 'Christopher',
  'education': 'PhD',
  'address': '789 Pine Ln'},
 {'name': 'Eve',
  'father_name': 'Michael',
  'education': 'High School',
  'address': '1011 Elm St'},
 {'name': 'William',
  'father_name': 'Daniel',
  '

In [ ]:
# prompt: Make a function with above list when we ask name as a input, it will give information to that person and recheck by LLM with last cell

import pandas as pd

def get_person_info(name, samples):
  """
  Retrieves information about a person from a list of dictionaries.

  Args:
    name: The name of the person to search for.
    samples: A list of dictionaries, where each dictionary represents a person and
             contains keys like "name", "father_name", "education", and "address".

  Returns:
    A dictionary containing the person's information if found, otherwise None.
  """
  for person in samples:
    if person["name"] == name:
      return person
  return None

def verify_with_llm(person_info):
  """
  Verifies the person's information using a large language model.

  Args:
    person_info: A dictionary containing the person's information.

  Returns:
    A string representing the LLM's verification.
  """

  # Construct prompt for LLM based on person's information
  prompt = f"""Verify the following information:\nName: {person_info['name']}\nFather's Name: {person_info['father_name']}\nEducation: {person_info['education']}\nAddress: {person_info['address']}
  \nIs this information likely to be correct?  Respond with 'Correct' or 'Incorrect' only"""

  # Replace with actual LLM call using the google.generativeai library
  # The following is a placeholder; uncomment and modify as needed
  # response = genai.generate_text(prompt=prompt, model='models/gemini-pro')
  # verification_result = response.result

  # Placeholder result (replace with actual LLM response)
  verification_result = "Correct" # Example: Replace with actual LLM output

  return verification_result

# Example usage
name_to_find = input("Enter the name: ")
person_info = get_person_info(name_to_find, samples)

if person_info:
  print(f"Information for {name_to_find}:")
  print(person_info)

  verification = verify_with_llm(person_info)
  print(f"\nLLM Verification: {verification}")

else:
  print(f"No information found for {name_to_find}.")

Enter the name: Alice
Information for Alice:
{'name': 'Alice', 'father_name': 'Robert', 'education': 'PhD', 'address': '789 Pine Ln'}

LLM Verification: Correct


## 2. LangChain Function/Tool Calling Calculator

In [ ]:
!pip install langchain langchain_community langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f1988a40-bb4e-425b-9b94-5608fc2fb0c1-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [ ]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b4b22658-c325-4e72-8476-f0518059182a-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

In [ ]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-98199979-e760-4288-8dd5-608f0c63b6fd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '1c4dd385-50a1-4fbd-8c1f-268ccb816911', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 3, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)


<ipython-input-9-f11f73b77070>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
# Query the agent
response1 = agent.run("What is 10 + 20?")  # Uses the tool
print(response1)



> Entering new AgentExecutor chain...
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Thought: I now know the final answer
Final Answer: 30

> Finished chain.
30
